In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.datasets import load_diabetes
import matplotlib.pyplot as plt
# import seaborn as sns

In [ ]:
class RidgeRegression:
  def __init__(self):
    self.w = None
  
  def fit(self, X, y, alpha):
    X = np.c_[np.ones(X.shape[0]), X]
    xTranspose = X.T
    A = xTranspose @ X
    I = np.identity(A.shape[0])
    c = xTranspose @ y
    self.w = np.linalg.inv(A + alpha * I) @ c
  
  def predict(self, X):
    X = np.c_[np.ones(X.shape[0]), X]
    return X @ self.w
    

In [ ]:
class LassoRegression:

  def soft_threshold(rho, alpha):
    if rho < alpha:
      return rho + alpha
    elif rho > alpha:
      return rho - alpha
    return 0
  
  def coordinate_descent(self,
                         X,
                         y,
                         theta,
                         alpha = 0.1,
                         n_iterations = 1000,
                         intercept = False):
    # The intercept parameter allows to specify whether or not we regularize theta_0
    X = np.array(X)
    y = np.array(y)
 
    m, n = X.shape
    X /= (np.linalg.norm(X, axis=0))

    for _ in range(n_iterations):
      for j in range(n):
        X_j = X[:,j].reshape(-1, 1)
        y_pred = X @ theta
        rho = X_j.T @ (y - y_pred + theta[j] * X_j)

        if intercept:
          if j == 0:
            theta[j] = rho
          else:
            theta[j] = self.soft_threshold(rho, alpha)
        else:
          theta[j] = self.soft_threshold(rho, alpha)
    return theta.flatten()
  
  def predict(self, X, theta):
    return X @ theta

In [ ]:
diabetes = load_diabetes()
# sns.heatmap(data.corr(), annot=True)
ridge = RidgeRegression()
lasso = LassoRegression()

alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
testMse = []
testR2 = []
coefficients = []

def alphaSweep():
  for a in alphas:
    ridge.fit(diabetes.data, diabetes.target, a)
    y_pred = ridge.predict(diabetes.data)
    testMse.append(mean_squared_error(diabetes.target, y_pred))
    testR2.append(r2_score(diabetes.target, y_pred))
    coefficients.append(ridge.w)
    print(f"[Ridge] Alpha: {a} | Mean Squared Error: {testMse[-1]} | R2 Score: {testR2[-1]}")

alphaSweep()

# Plotting Ridge Regression Metrics
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(21, 5))
ax1.plot(alphas, testMse, marker='o', markersize=4)
ax1.set_xscale('log')
ax1.set_xlabel('α')
ax1.set_ylabel('MSE (%)')
ax1.set_title('MSE vs Alpha')
ax1.grid(True, alpha = 0.3)

ax2.plot(alphas, testR2, marker='o', markersize=4, color='green')
ax2.set_xscale('log')
ax2.set_xlabel('α')
ax2.set_ylabel('R2 Score')
ax2.set_title('R2 Score vs Alpha')
ax2.grid(True, alpha = 0.3)

coefficients = np.array(coefficients[1:])
ax3.plot(alphas[1:], np.array(coefficients), marker='o', markersize=4)
ax3.set_xscale('log')
ax3.set_xlabel('α')
ax3.set_ylabel('Coefficient Values')
ax3.set_title('Coefficient Values vs Alpha')
ax3.grid(True, alpha = 0.3)

fig.delaxes(ax4)

plt.tight_layout()
plt.savefig('../images/ridge_metrics.png')
plt.show()
"""
theta = lasso.coordinate_descent(diabetes.data, diabetes.target, np.zeros(diabetes.data.shape[1]), alpha=0.1, n_iterations=1000, intercept = True)
y_pred_lasso = lasso.predict(diabetes.data, theta)
lasso_mse = mean_squared_error(diabetes.target, y_pred_lasso)
lasso_r2 = r2_score(diabetes.target, y_pred_lasso)
print("[Lasso] Mean Squared Error:", lasso_mse)
print("[Lasso] R2 Score:", lasso_r2)
"""